In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
sns.set()

### Read and filter data

In [ ]:
# Get Covid Stringency Index data
stringency = pd.read_csv('covid-stringency-index.csv')
fin_strin = stringency[stringency['Entity'] == 'Finland']
fin_strin = fin_strin.drop(['Entity', 'Code'], axis=1)
fin_strin['Day'] = pd.to_datetime(fin_strin['Day'], format='%Y-%m-%d') 
start = fin_strin.iloc[0]['Day']
end = fin_strin.iloc[-1]['Day']
print(end)

#Get Finland covid data
global_covid = pd.read_csv('WHO-COVID-19-global-data.csv')
fin_covid = global_covid[global_covid['Country'] == 'Finland']
fin_covid = fin_covid.drop(['Country_code', 'Country', 'WHO_region', 'Cumulative_cases','Cumulative_deaths'], axis=1)
fin_covid['Date_reported'] = pd.to_datetime(fin_covid['Date_reported'], format='%Y-%m-%d' )

# Filter data and combine data
mask = (fin_covid['Date_reported'] >= start) & (fin_covid['Date_reported'] <= end)
fin_covid = fin_covid.loc[mask]

## Insert this to  col 2
strin_values = fin_strin.loc[:, 'stringency_index'].to_numpy()
days = [i for i in range(fin_covid.shape[0])]
fin_covid.insert(0, 'Day', days)
fin_covid.insert(1, 'Stringency_index', strin_values)

# Final datapoint
fin_covid = fin_covid.drop(['Date_reported'], axis=1)

print("\n", fin_covid.describe())
fin_covid.tail()


### Plotting data

In [ ]:
fig1= plt.figure(1, figsize=(10,10))
plt.xlabel('Day')
plt.ylabel('Stringency_index')
plt.plot(fin_covid['Day'], fin_covid['Stringency_index'], '+', color='blue')
plt.show()

fig2 = plt.figure(2, figsize=(10,10))
plt.xlabel('Day')
plt.ylabel('New_cases')
plt.plot(fin_covid['Day'], fin_covid['New_cases'], '+', color='green')
plt.show() 

fig3 = plt.figure(3, figsize=(10,10))
plt.xlabel('Day')
plt.ylabel('New_deaths')
plt.plot(fin_covid['Day'], fin_covid['New_deaths'], color='purple')
plt.show()


In [ ]:
sns.pairplot(fin_covid)